# TODO:
#### Automatic data retrieval from google forms for during-suvrvey checking
[X] spreadsheets url inference <br>
[X] gather results in python script

#### Qualification filtering
[ ] ~50 qualifications

#### Frauder detection
[X] > X ratio repeated ===> discarded

#### Honeypots
[X] place honeypots from a defined set in each form<br>
[X] honeypot frauders detection

#### MTurk templating
[ ] Create guidelines for the workers

#### Gform templating
[X] Remove index of the emojis and add separator btwn each question NB: not practical

#### Emojis png
[X] Rerun creation of emojis png + upload on drive


In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from src.data.auto_mturk import create_mturk_client,monitor_worker_tags,clean_own_worker,Turker
from src.data.auto_drive import get_drive_service, download_drive_txt, download_all_csv_results
from src.data.fraudulous import detect_fraud_worker
from src.utils import generate_password 
from src.constants import TOKEN_PATH,CREDS_PATH, URL_INDEX_PATH, AWS_KEYS_PATH, FORMS_RESULTS_DIR,HONEYPOTS
from src.utils import read_access_keys
import pandas as pd
import jdc
import pickle as pk

## Urls info retrieval

In [26]:
############ TODO: copy-paste the id of the most recent version of the url file ############
file_id = "1Y5qLoVQmkT16zO-yUX55o3FlQelyfRYb9iQN7xdCuNU"
############################################################################################
# Retrieve the urls of the forms generated by the app script
service = get_drive_service()
download_drive_txt(URL_INDEX_PATH,file_id,service)  # download the most recent url_index
url_index = pd.read_csv(URL_INDEX_PATH,sep=",",header=None,names=['url','res_url'])

Download 100%


In [31]:
download_all_csv_results(url_index,FORMS_RESULTS_DIR,service)

Download 100%
Download 100%


In [32]:
form0_df = pd.read_csv(FORMS_RESULTS_DIR.joinpath("0.csv"))

In [33]:
form0_df

,Timestamp,Worker ID,#️⃣,*️⃣,©️,®️,‼️,⁉️,™️,ℹ️,↔️,↕️,↖️
0,26/11/2020 11:01:46,FRAUDERREPEAT0,asd,asd,asd,asd,surprised,asd,tm,asd,asd,asd,asd
1,26/11/2020 11:02:19,FRAUDERHONEY0,a,b,c,d,e,f,g,h,i,j,k
2,26/11/2020 11:03:25,HONEST0,lk,asdf,pej,asdfn,surprise,what,tm,d,i,p,dfe


In [34]:
detect_fraud_worker(form0_df)

['FRAUDERREPEAT0']

## Honeypots

In [2]:
from src.constants import HONEYPOTS
import Levenshtein
import pickle as pk

In [1]:
from IPython.core.debugger import set_trace

In [54]:
detect_honey_frauders(form0_df,HONEYPOTS)

['FRAUDERHONEY0']

In [59]:
dic = pk.load(open("../data/processed/emojis_png/all/dic.pk","rb"))

In [63]:
for i in [dic[em] for em in honeypots.keys()]:
    print(i,end=",")

3006,3091,2897,1244,2282,451,2283,2280,613,387,740,428,597,3059,1284,444,2528,2640,2885,1095,2533,828,699,2979,827,

In [3]:
HONEYPOTS

{'☂️': ['rain', 'umbrella'],
 '⭐': ['star'],
 '☀️': ['sun', 'sunny'],
 '🥝': ['kiwi'],
 '🥜': ['peanut', 'peanuts'],
 '🏀': ['basket', 'basketball'],
 '🍐': ['pear'],
 '🍑': ['peach'],
 '🍒': ['cherry'],
 '🥑': ['avocado'],
 '🥒': ['pickle'],
 '🥓': ['bacon'],
 '🥕': ['carrot'],
 '🍉': ['watermelon'],
 '🍋': ['lemon', 'lime'],
 '🍌': ['banana'],
 '🍍': ['ananas'],
 '🍎': ['apple'],
 '🍓': ['strawberry'],
 '🍔': ['burger'],
 '🍕': ['pizza'],
 '🍩': ['donut', 'donuts'],
 '🍪': ['cookie', 'cookies'],
 '™️': ['tm'],
 '‼️': ['surprised']}

## MTurk management

In [70]:
turk = Turker(hitlayout="3QS25971A2UW6PKAU5WN9771C2JXNI",
             MaxAssignments = 10,
             LifetimeInSeconds = 600,
             AutoApprovalDelayInSeconds=600,
             AssignmentDurationInSeconds=600,
             Reward='0.01',)

Loading hit2form


In [72]:
turk.create_forms_hits(url_index)

Creating hit for form 0
Creating hit for form 1
Creating hit for form 2
Creating hit for form 3


In [77]:
turk.list_hits()

Expiration:Nov 23 2020 10:48:35 (5.13 minutes left)


,HITId,Status,Completed,Percent_completed
FormIdx,,,,
0,3W0XM68YZPG4RN28L5YV3JOY3OEK1E,Assignable,1,10
1,3X52SWXE0XQF9Q6NQG6I8Q1KUSECWS,Assignable,0,0
2,3Y3CZJSZ9KEP29GMCC947LNTAWDR5W,Assignable,0,0
3,3CRWSLD91KPKD39CCJ9MUSTAL4JOMA,Assignable,1,10


In [74]:
clean_own_worker(turk.client)

Worker already clean


In [75]:
turk.get_url("3W0XM68YZPG4RN28L5YV3JOY3OEK1E")

'https://workersandbox.mturk.com/mturk/preview?groupId=3BW6RX62GFQF0UCE0YFLZL7NB2LLEY'

In [76]:
monitor_worker_tags(turk.client)

searching..
Tagging worker A29C1XYH77RQYM


KeyboardInterrupt: 

In [80]:
turk.list_all_results()

No results ready yet for 3X52SWXE0XQF9Q6NQG6I8Q1KUSECWS
No results ready yet for 3Y3CZJSZ9KEP29GMCC947LNTAWDR5W


,WorkerId,HITId,Answer,Code,Status
0,A29C1XYH77RQYM,3CRWSLD91KPKD39CCJ9MUSTAL4JOMA,190,190,Submitted
0,A29C1XYH77RQYM,3W0XM68YZPG4RN28L5YV3JOY3OEK1E,cako,932,Submitted


In [ ]:
turk.list_results("")

In [79]:
turk.approve_correct_hits()

In [82]:
# reset the hits
turk.approve_all_hits()
turk.stop_all_hits()
turk.delete_all_hits()

Approving assignment 3DPNQGW4LM0H6RBWWLSYFHMZEP164T
Approving assignment 39U1BHVTDMC2QT34SKV8GMM7KXG3TG
Deleting hit 3CRWSLD91KPKD39CCJ9MUSTAL4JOMA
Deleting hit 3W0XM68YZPG4RN28L5YV3JOY3OEK1E


## Qualifications

In [ ]:
# Create the qualification we need to distinguish workers who already performed an emoji task
turk.client.create_qualification_type(
    Name='emojidone',
    Description="Qualification to recognize workers that already performed an emoji-related task",
    QualificationTypeStatus='Active',
    AutoGranted=True,
    AutoGrantedValue=1
)
# QualificationTypeId='3OR1BBO28PIVPWZMRDTWE8U6OZXNGN